Needed imports and utility

In [ ]:
import os
from ansys.speos.core.proto_message_utils import protobuf_message_to_str
from ansys.speos.core.job import JobFactory
from ansys.speos.core.geometry_utils import AxisSystem
from ansys.speos.core.client import SpeosClient

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")
speos_file = os.path.join(tests_data_path, os.path.join("LG_50M_Colorimetric_short.sv5", "LG_50M_Colorimetric_short.sv5"))

def print_message(msg):
    print(protobuf_message_to_str(msg))

def clean_all_dbs(speos_client: SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

Create connection with speos rpc server

In [ ]:
from ansys.speos.core.speos import Speos
speos = Speos(host="localhost", port=50051)

Create empty scene and load file

In [ ]:
my_scene = speos.client.scenes().create()
my_scene.load_file(file_uri=speos_file)

Retrieve source ray paths for the first source

In [ ]:
for ray_path in my_scene.get_source_ray_paths(source_path=my_scene.get().sources[0].name, rays_nb=2):
    print_message(ray_path)

Create a job for first simu

In [ ]:
my_job = speos.client.jobs().create(message=JobFactory.new(name="my_job",
                                                           scene=my_scene,
                                                           simulation_path=my_scene.get().simulations[0].name,
                                                           properties=JobFactory.direct_mc_props()))

my_job.start()

In [ ]:
my_job.get_state()

In [ ]:
my_job.get_results()

In [ ]:
my_job.delete()

In [ ]:
clean_all_dbs(speos.client)